In [1]:
import pandas as pd
import requests
import os
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [2]:
for dir_name in ['data', 'data/gameweek_scores', 'data/match_stats', 'data/player_stats']:
    os.makedirs(dir_name, exist_ok=True)

base_url = "https://api.sofascore.com/api/v1/unique-tournament/17/season/41886/"
current_gw = 5

### Team

##### Overall

In [3]:
r = requests.get(base_url + "standings/total").json()
stats = r['standings'][0]['rows']

team_stats_overall = pd.DataFrame([{
    'name': team['team']['name'],
    'position': team['position'],
    'matches': team['matches'],
    'wins': team['wins'],
    'losses': team['losses'],
    'draws': team['draws'],
    'scoresFor': team['scoresFor'],
    'scoresAgainst': team['scoresAgainst'],
    'points': team['points']
} for team in stats])

##### Home

In [4]:
r = requests.get(base_url + "standings/home").json()
stats = r['standings'][0]['rows']

team_stats_home = pd.DataFrame([{
    'name': team['team']['name'],
    'position': team['position'],
    'matches': team['matches'],
    'wins': team['wins'],
    'losses': team['losses'],
    'draws': team['draws'],
    'scoresFor': team['scoresFor'],
    'scoresAgainst': team['scoresAgainst'],
    'points': team['points']
} for team in stats])

##### Away

In [5]:
r = requests.get(base_url + "standings/away").json()
stats = r['standings'][0]['rows']

team_stats_away = pd.DataFrame([{
    'name': team['team']['name'],
    'position': team['position'],
    'matches': team['matches'],
    'wins': team['wins'],
    'losses': team['losses'],
    'draws': team['draws'],
    'scoresFor': team['scoresFor'],
    'scoresAgainst': team['scoresAgainst'],
    'points': team['points']

} for team in stats])

In [22]:
team_stats_away

,name,position,matches,wins,losses,draws,scoresFor,scoresAgainst,points
0,Arsenal,1,2,2,0,0,5,0,6
1,Brighton & Hove Albion,2,3,2,1,0,5,3,6
2,Tottenham Hotspur,3,3,1,0,2,5,3,5
3,Manchester City,4,2,1,0,1,5,3,4
4,West Ham United,5,2,1,1,0,1,1,3
5,Southampton,6,2,1,1,0,3,5,3
6,Chelsea,7,3,1,2,0,2,5,3
7,Manchester United,8,2,1,1,0,1,4,3
8,Brentford,9,3,0,1,2,5,6,2
9,Everton,10,3,0,1,2,3,4,2


### Gameweek

In [6]:
def get_gameweek_matches(gw_no):
    try:
        gw_score = pd.read_json('data/gameweek_scores/scores_gw{}.json'.format(gw_no))
    except ValueError:
        r = requests.get(base_url + "events/round/{}".format(gw_no)).json()
        gw_score = pd.DataFrame([{
            'gameWeek': gw_no,
            'matchId': event['id'],
            'homeTeam': event['homeTeam']['name'],
            'homeScore': event['homeScore']['current'],
            'awayScore': event['awayScore']['current'],
            'awayTeam': event['awayTeam']['name'],
            'scoreCombined': '{}-{}'.format(event['homeScore']['current'], event['awayScore']['current'])
        } for event in r['events'] if event['status']['code'] == 100])
        with open('data/gameweek_scores/scores_gw{}.json'.format(gw_no), 'w') as f:
            gw_score.to_json(f, orient='records')
    return gw_score

gw_scores_all = pd.concat([get_gameweek_matches(gw_no) for gw_no in range(1, current_gw+1)])

### Match

In [7]:
def get_match_stats(row):
    match_id = row['matchId']
    try:
        df = pd.read_json('data/match_stats/stats_match{}.json'.format(match_id))
    except ValueError:
        r = requests.get('https://api.sofascore.com/api/v1/event/{}/statistics'.format(match_id)).json()
        stats = []
        for x in r['statistics'][0]['groups']:
            if x['groupName'] not in ['TVData', 'Passes', 'Duels']:
                stats.extend([{
                    'category': x['groupName'],
                    'name': y['name'],
                    'home': y['home'],
                    'away': y['away']
                } for y in x['statisticsItems']])
        df = pd.DataFrame(stats)
        for k, v in row.items():
            df[k] = v
        with open('data/match_stats/stats_match{}.json'.format(match_id), 'w') as f:
            df.to_json(f, orient='records')
    return df[['gameWeek', 'matchId', 'homeTeam', 'homeScore', 'awayScore', 'awayTeam', 'category', 'name', 'home', 'away']]

team_stats_scores_gw_all = pd.concat([get_match_stats(r) for i, r in gw_scores_all.iterrows()])

### Player

In [8]:
def get_player_stats_in_match(row):
    try:
        df = pd.read_json('data/player_stats/stats_players_match{}.json'.format(row['matchId']))
    except:
        r = requests.get('https://api.sofascore.com/api/v1/event/{}/lineups'.format(row['matchId'])).json()
        player_details = []
        for loc in ['home', 'away']:
            for player in r[loc]['players']:
                d = row.to_dict()
                d.update({
                    'playerId': player['player']['id'],
                    'playerName': player['player']['name'],
                    'teamName': row['{}Team'.format(loc)],
                    'playerPosition': player['player']['position'],
                    'playingOn': loc.title(),
                    'subsititute': player['substitute']
                })
                d.update(player['statistics'])
                player_details.append(d)
        df = pd.DataFrame(player_details)
        with open('data/player_stats/stats_players_match{}.json'.format(row['matchId']), 'w') as f:
            df.to_json(f, orient='records')
    return df

player_stats_gw_all = pd.concat([get_player_stats_in_match(row) for i, row in gw_scores_all.iterrows()])

### Player Statistics

In [9]:
COLUMNS_G = [
    'gameWeek', 'homeTeam', 'homeScore', 'awayScore', 'awayTeam', 'playerName', 'teamName', 'playerPosition',
    'playingOn', 'subsititute', 'minutesPlayed', 'rating', 'actualFPLPoints', 'possibleFPLPoints',
    'savedShotsFromInsideTheBox', 'saves', 'totalKeeperSweeper', 'accurateKeeperSweeper', 'possessionLostCtrl',
    'keyPass', 'aerialWon', 'aerialLost', 'duelWon', 'duelLost', 'interceptionWon', 'wasFouled', 'fouls',
    'bigChanceCreated', 'goodHighClaim', 'punches', 'goalAssist', 'ownGoals', 'errorLeadToAGoal',
    'errorLeadToAShot', 'penaltySave', 'penaltyConceded'
]
COLUMNS_NG = [
    'gameWeek', 'homeTeam', 'homeScore', 'awayScore', 'awayTeam', 'playerName', 'teamName', 'playerPosition',
    'playingOn', 'subsititute', 'minutesPlayed', 'rating', 'actualFPLPoints', 'possibleFPLPoints', 'goals',
    'goalAssist', 'goalThreat', 'assistThreat', 'hitWoodwork', 'shotOffTarget', 'bigChanceMissed',
    'onTargetScoringAttempt', 'penaltyWon', 'penaltyMiss', 'bigChanceCreated', 'keyPass', 'touches',
    'accurateCross', 'totalCross', 'totalOffside', 'wasFouled', 'possessionLostCtrl', 'aerialWon',
    'aerialLost', 'duelWon', 'duelLost', 'dispossessed', 'totalContest', 'wonContest', 'clearanceOffLine',
    'interceptionWon', 'totalTackle', 'lastManTackle', 'fouls', 'blockedScoringAttempt', 'challengeLost',
    'ownGoals', 'errorLeadToAGoal', 'errorLeadToAShot', 'penaltyConceded'
]

In [10]:
def calculate_actual_attack_points(row):
    pts = row['goalAssist'] * 3
    if row['playerPosition'] == 'F':
        pts += (row['goals'] * 4)
    elif row['playerPosition'] == 'M':
        pts += (row['goals'] * 5)
    elif row['playerPosition'] == 'D':
        pts += (row['goals'] * 6)
    return pts

def calculate_possible_attack_points(row):
    pts = row['assistThreat'] * 3
    if row['playerPosition'] == 'F':
        pts += (row['goalThreat'] * 4)
    elif row['playerPosition'] == 'M':
        pts += (row['goalThreat'] * 5)
    elif row['playerPosition'] == 'D':
        pts += (row['goalThreat'] * 6)
    return pts

In [11]:
def get_player_stats(stats, playerPosition='NG', minutesPlayed=1, gameweeks=[], matchId=None):
    df = stats.fillna(0)
    df['goalThreat'] = df['goals'] + df['hitWoodwork'] + df['bigChanceMissed']
    df['assistThreat'] = df['goalAssist'] + df['penaltyWon'] + df['bigChanceCreated']
    df['actualFPLPoints'] = df.apply(lambda x: calculate_actual_attack_points(x), axis=1)
    df['possibleFPLPoints'] = df.apply(lambda x: calculate_possible_attack_points(x), axis=1)
    df = df[df.minutesPlayed >= minutesPlayed]
    df = df[df.playerPosition == playerPosition] if playerPosition != 'NG' else df[df.playerPosition != 'G']
    if gameweeks:
        df = df[df['gameWeek'].isin(gameweeks)]
    if matchId:
        df = df[df.matchId == matchId]
    return df[COLUMNS_G] if playerPosition == 'G' else df[COLUMNS_NG]

In [12]:
players_f = get_player_stats(player_stats_gw_all, 'F')  # Forwards
players_m = get_player_stats(player_stats_gw_all, 'M')  # Mid-fielders
players_d = get_player_stats(player_stats_gw_all, 'D')  # Defenders
players_g = get_player_stats(player_stats_gw_all, 'G')  # Goal Keepers
players_ng = get_player_stats(player_stats_gw_all)  # All NG Players

In [13]:
def stats_filter(data, playerName=None, playerPosition=None, teamName=None, location='All', gameWeeks=[]):
    filtered = data.copy()
    if playerName:
        filtered = filtered[filtered['playerName'] == playerName]
    if playerPosition:
        filtered = filtered[filtered['playerPosition'] == playerPosition]
    if teamName:
        filtered = filtered[filtered['teamName'] == teamName]
    if location != 'All':
        filtered = filtered[filtered['playingOn'] == location]
    if gameWeeks:
        filtered = filtered[filtered['gameWeek'].isin(gameWeeks)]
    return filtered

### Players w.r.t. position on Home and/or Away

In [29]:
selected_position = 'F'

##### Home and Away

In [26]:
filtered = stats_filter(players_ng, playerPosition=selected_position)
filtered.groupby(['playerName']).agg(sum).sort_values(['possibleFPLPoints', 'actualFPLPoints', 'rating', 'minutesPlayed'], ascending=False).iloc[:50]

,gameWeek,homeScore,awayScore,subsititute,minutesPlayed,rating,actualFPLPoints,possibleFPLPoints,goals,goalAssist,goalThreat,assistThreat,hitWoodwork,shotOffTarget,bigChanceMissed,onTargetScoringAttempt,penaltyWon,penaltyMiss,bigChanceCreated,keyPass,touches,accurateCross,totalCross,totalOffside,wasFouled,possessionLostCtrl,aerialWon,aerialLost,duelWon,duelLost,dispossessed,totalContest,wonContest,clearanceOffLine,interceptionWon,totalTackle,lastManTackle,fouls,blockedScoringAttempt,challengeLost,ownGoals,errorLeadToAGoal,errorLeadToAShot,penaltyConceded
playerName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Gabriel Martinelli,15,8,9,0,435.0,37.7,15.0,39.0,3.0,0.0,6.0,3.0,1.0,7.0,2.0,7.0,0.0,0.0,3.0,12.0,257.0,4.0,28.0,1.0,8.0,68.0,2.0,4.0,27.0,29.0,9.0,17.0,11.0,0.0,2.0,6.0,0.0,7.0,5.0,3.0,0.0,0.0,0.0,0.0
Kevin De Bruyne,15,17,7,1,378.0,39.0,14.0,39.0,1.0,3.0,3.0,8.0,0.0,3.0,2.0,5.0,0.0,0.0,5.0,15.0,323.0,12.0,38.0,0.0,5.0,82.0,0.0,4.0,15.0,15.0,3.0,8.0,5.0,0.0,0.0,5.0,0.0,3.0,5.0,2.0,0.0,0.0,0.0,0.0
Pascal Groß,15,4,5,0,450.0,37.6,18.0,32.0,3.0,1.0,4.0,4.0,0.0,2.0,1.0,4.0,0.0,0.0,3.0,9.0,260.0,5.0,30.0,2.0,5.0,87.0,1.0,10.0,14.0,19.0,6.0,0.0,0.0,0.0,2.0,8.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0
Luis Díaz,15,16,5,0,438.0,36.4,15.0,30.0,3.0,0.0,6.0,0.0,1.0,5.0,2.0,7.0,0.0,0.0,0.0,2.0,296.0,1.0,4.0,1.0,10.0,72.0,4.0,12.0,36.0,47.0,19.0,22.0,14.0,0.0,2.0,8.0,0.0,7.0,2.0,2.0,0.0,0.0,0.0,0.0
Jack Harrison,15,9,4,0,420.0,36.8,14.0,26.0,1.0,3.0,1.0,7.0,0.0,1.0,0.0,3.0,0.0,0.0,4.0,16.0,314.0,9.0,34.0,1.0,9.0,117.0,1.0,6.0,32.0,43.0,15.0,23.0,12.0,0.0,4.0,10.0,0.0,7.0,1.0,5.0,0.0,0.0,0.0,0.0
Anthony Elanga,10,7,4,2,156.0,20.0,3.0,24.0,0.0,1.0,3.0,3.0,1.0,1.0,2.0,1.0,0.0,0.0,2.0,5.0,70.0,0.0,0.0,2.0,3.0,22.0,5.0,1.0,11.0,9.0,2.0,8.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bernardo Silva,15,17,7,2,307.0,37.0,16.0,22.0,2.0,2.0,2.0,4.0,0.0,1.0,0.0,3.0,0.0,0.0,2.0,9.0,229.0,1.0,6.0,0.0,4.0,34.0,0.0,1.0,16.0,14.0,3.0,10.0,6.0,0.0,1.0,6.0,0.0,3.0,2.0,3.0,0.0,0.0,0.0,0.0
Mathias Jensen,15,11,6,0,362.0,36.7,11.0,22.0,1.0,2.0,2.0,4.0,1.0,2.0,0.0,1.0,0.0,0.0,2.0,10.0,279.0,8.0,28.0,0.0,3.0,87.0,4.0,0.0,18.0,14.0,2.0,3.0,0.0,0.0,0.0,11.0,0.0,2.0,1.0,7.0,0.0,0.0,1.0,0.0
Granit Xhaka,15,8,9,0,448.0,35.7,11.0,21.0,1.0,2.0,3.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,10.0,326.0,0.0,6.0,1.0,7.0,58.0,7.0,2.0,18.0,13.0,5.0,2.0,1.0,0.0,3.0,3.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0


##### Home

In [27]:
filtered = stats_filter(players_ng, playerPosition=selected_position, location='Home')
filtered.groupby(['playerName']).agg(sum).sort_values(['possibleFPLPoints', 'actualFPLPoints', 'minutesPlayed'], ascending=False).iloc[:50]

,gameWeek,homeScore,awayScore,subsititute,minutesPlayed,rating,actualFPLPoints,possibleFPLPoints,goals,goalAssist,goalThreat,assistThreat,hitWoodwork,shotOffTarget,bigChanceMissed,onTargetScoringAttempt,penaltyWon,penaltyMiss,bigChanceCreated,keyPass,touches,accurateCross,totalCross,totalOffside,wasFouled,possessionLostCtrl,aerialWon,aerialLost,duelWon,duelLost,dispossessed,totalContest,wonContest,clearanceOffLine,interceptionWon,totalTackle,lastManTackle,fouls,blockedScoringAttempt,challengeLost,ownGoals,errorLeadToAGoal,errorLeadToAShot,penaltyConceded
playerName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Gabriel Martinelli,11,8,4,0,270.0,23.9,10.0,24.0,2.0,0.0,3.0,3.0,1.0,3.0,0.0,5.0,0.0,0.0,3.0,11.0,177.0,3.0,21.0,1.0,5.0,47.0,1.0,3.0,18.0,19.0,4.0,12.0,7.0,0.0,2.0,5.0,0.0,6.0,5.0,1.0,0.0,0.0,0.0,0.0
Luis Díaz,11,12,2,0,270.0,23.6,15.0,20.0,3.0,0.0,4.0,0.0,0.0,3.0,1.0,6.0,0.0,0.0,0.0,2.0,195.0,1.0,3.0,1.0,5.0,41.0,1.0,8.0,24.0,25.0,6.0,18.0,11.0,0.0,2.0,7.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0
Jack Harrison,9,6,2,0,262.0,22.6,11.0,20.0,1.0,2.0,1.0,5.0,0.0,1.0,0.0,3.0,0.0,0.0,3.0,11.0,214.0,7.0,27.0,1.0,6.0,87.0,0.0,4.0,18.0,31.0,11.0,17.0,8.0,0.0,3.0,4.0,0.0,5.0,0.0,3.0,0.0,0.0,0.0,0.0
Reece James,6,4,3,0,180.0,15.8,8.0,19.0,1.0,1.0,2.0,3.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,3.0,174.0,3.0,9.0,0.0,2.0,20.0,4.0,2.0,12.0,6.0,1.0,2.0,2.0,0.0,1.0,4.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0
Granit Xhaka,11,8,4,0,270.0,21.4,8.0,18.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,6.0,200.0,0.0,6.0,1.0,3.0,39.0,3.0,1.0,7.0,9.0,4.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0
Kevin De Bruyne,11,14,2,1,200.0,23.3,8.0,16.0,1.0,1.0,2.0,2.0,0.0,1.0,1.0,3.0,0.0,0.0,1.0,8.0,191.0,8.0,26.0,0.0,2.0,47.0,0.0,3.0,8.0,8.0,2.0,5.0,4.0,0.0,0.0,2.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0
Pascal Groß,6,1,0,0,180.0,14.9,5.0,16.0,1.0,0.0,2.0,2.0,0.0,2.0,1.0,1.0,0.0,0.0,2.0,6.0,96.0,2.0,15.0,0.0,1.0,35.0,1.0,5.0,4.0,9.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
Neeskens Kebano,6,4,3,0,156.0,13.3,3.0,16.0,0.0,1.0,2.0,2.0,1.0,3.0,1.0,0.0,0.0,0.0,1.0,3.0,66.0,0.0,2.0,2.0,2.0,18.0,0.0,3.0,8.0,13.0,3.0,6.0,3.0,0.0,0.0,3.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0
Anthony Elanga,4,3,3,1,46.0,6.8,3.0,16.0,0.0,1.0,2.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,20.0,0.0,0.0,1.0,1.0,7.0,1.0,0.0,3.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Away

In [28]:
filtered = stats_filter(players_ng, playerPosition=selected_position, location='Away')
filtered.groupby(['playerName']).agg(sum).sort_values(['possibleFPLPoints', 'actualFPLPoints', 'minutesPlayed'], ascending=False).iloc[:50]

,gameWeek,homeScore,awayScore,subsititute,minutesPlayed,rating,actualFPLPoints,possibleFPLPoints,goals,goalAssist,goalThreat,assistThreat,hitWoodwork,shotOffTarget,bigChanceMissed,onTargetScoringAttempt,penaltyWon,penaltyMiss,bigChanceCreated,keyPass,touches,accurateCross,totalCross,totalOffside,wasFouled,possessionLostCtrl,aerialWon,aerialLost,duelWon,duelLost,dispossessed,totalContest,wonContest,clearanceOffLine,interceptionWon,totalTackle,lastManTackle,fouls,blockedScoringAttempt,challengeLost,ownGoals,errorLeadToAGoal,errorLeadToAShot,penaltyConceded
playerName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Kevin De Bruyne,4,3,5,0,178.0,15.7,6.0,23.0,0.0,2.0,1.0,6.0,0.0,2.0,1.0,2.0,0.0,0.0,4.0,7.0,132.0,4.0,12.0,0.0,3.0,35.0,0.0,1.0,7.0,7.0,1.0,3.0,1.0,0.0,0.0,3.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0
Pascal Groß,9,3,5,0,270.0,22.7,13.0,16.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,3.0,164.0,3.0,15.0,2.0,4.0,52.0,0.0,5.0,10.0,10.0,5.0,0.0,0.0,0.0,2.0,6.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
Gabriel Martinelli,4,0,5,0,165.0,13.8,5.0,15.0,1.0,0.0,3.0,0.0,0.0,4.0,2.0,2.0,0.0,0.0,0.0,1.0,80.0,1.0,7.0,0.0,3.0,21.0,1.0,1.0,9.0,10.0,5.0,5.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0
James Ward-Prowse,4,5,3,0,180.0,15.7,8.0,14.0,1.0,1.0,1.0,3.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,8.0,111.0,6.0,15.0,0.0,1.0,24.0,1.0,2.0,6.0,5.0,1.0,0.0,0.0,0.0,1.0,4.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
Anthony Gordon,11,4,3,0,270.0,21.5,10.0,13.0,2.0,0.0,2.0,1.0,0.0,0.0,0.0,7.0,0.0,0.0,1.0,2.0,129.0,0.0,2.0,2.0,4.0,39.0,1.0,10.0,13.0,31.0,3.0,8.0,1.0,0.0,0.0,7.0,0.0,6.0,2.0,5.0,0.0,0.0,0.0,0.0
Eberechi Eze,6,5,3,0,169.0,13.9,6.0,12.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,84.0,1.0,3.0,1.0,4.0,25.0,1.0,4.0,13.0,13.0,6.0,11.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bernardo Silva,4,3,5,1,102.0,15.0,8.0,11.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,75.0,0.0,2.0,0.0,2.0,14.0,0.0,0.0,7.0,6.0,2.0,3.0,1.0,0.0,0.0,4.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0
Rico Henry,9,6,5,0,270.0,19.4,3.0,11.0,0.0,1.0,1.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0,175.0,1.0,5.0,0.0,4.0,47.0,5.0,4.0,12.0,12.0,3.0,4.0,1.0,0.0,2.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
Alexis Mac Allister,9,3,5,0,270.0,21.4,10.0,10.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,181.0,0.0,1.0,0.0,2.0,28.0,2.0,1.0,13.0,15.0,6.0,2.0,2.0,0.0,7.0,7.0,0.0,4.0,1.0,4.0,1.0,0.0,0.0,0.0


### Selected player with respect to Home and/or Away

In [209]:
selected_player = 'Erling Haaland'

##### Home and Away

In [216]:
filtered = stats_filter(players_ng, playerName=selected_player)
filtered.sort_values(['playerPosition', 'playerName', 'gameWeek'], ascending=[True, True, False])

,gameWeek,homeTeam,homeScore,awayScore,awayTeam,playerName,teamName,playerPosition,subsititute,minutesPlayed,rating,goals,goalAssist,hitWoodwork,shotOffTarget,bigChanceMissed,onTargetScoringAttempt,penaltyWon,penaltyMiss,bigChanceCreated,keyPass,touches,accurateCross,totalCross,totalOffside,wasFouled,possessionLostCtrl,aerialWon,aerialLost,duelWon,duelLost,dispossessed,totalContest,wonContest,clearanceOffLine,interceptionWon,totalTackle,lastManTackle,fouls,blockedScoringAttempt,challengeLost,ownGoals,errorLeadToAGoal,errorLeadToAShot,penaltyConceded
10,4,Manchester City,4,2,Crystal Palace,Erling Haaland,Manchester City,F,False,84.0,9.3,3.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,1.0,0.0,3.0,1.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
29,3,Newcastle United,3,3,Manchester City,Erling Haaland,Manchester City,F,False,90.0,7.1,1.0,0.0,1.0,1.0,2.0,3.0,0.0,0.0,1.0,1.0,33.0,0.0,0.0,0.0,2.0,9.0,0.0,2.0,5.0,4.0,1.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,2,Manchester City,4,0,AFC Bournemouth,Erling Haaland,Manchester City,F,False,74.0,7.3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,8.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
29,1,West Ham United,0,2,Manchester City,Erling Haaland,Manchester City,F,False,78.0,7.7,2.0,0.0,0.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


##### Home

In [218]:
filtered = stats_filter(players_ng, playerName=selected_player, location='Home')
filtered.sort_values(['playerPosition', 'playerName', 'gameWeek'], ascending=[True, True, False])

,gameWeek,homeTeam,homeScore,awayScore,awayTeam,playerName,teamName,playerPosition,subsititute,minutesPlayed,rating,goals,goalAssist,hitWoodwork,shotOffTarget,bigChanceMissed,onTargetScoringAttempt,penaltyWon,penaltyMiss,bigChanceCreated,keyPass,touches,accurateCross,totalCross,totalOffside,wasFouled,possessionLostCtrl,aerialWon,aerialLost,duelWon,duelLost,dispossessed,totalContest,wonContest,clearanceOffLine,interceptionWon,totalTackle,lastManTackle,fouls,blockedScoringAttempt,challengeLost,ownGoals,errorLeadToAGoal,errorLeadToAShot,penaltyConceded
10,4,Manchester City,4,2,Crystal Palace,Erling Haaland,Manchester City,F,False,84.0,9.3,3.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,1.0,0.0,3.0,1.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
10,2,Manchester City,4,0,AFC Bournemouth,Erling Haaland,Manchester City,F,False,74.0,7.3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,8.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Away

In [220]:
filtered = stats_filter(players_ng, playerName=selected_player, location='Away')
filtered.sort_values(['playerPosition', 'playerName', 'gameWeek'], ascending=[True, True, False])

,gameWeek,homeTeam,homeScore,awayScore,awayTeam,playerName,teamName,playerPosition,subsititute,minutesPlayed,rating,goals,goalAssist,hitWoodwork,shotOffTarget,bigChanceMissed,onTargetScoringAttempt,penaltyWon,penaltyMiss,bigChanceCreated,keyPass,touches,accurateCross,totalCross,totalOffside,wasFouled,possessionLostCtrl,aerialWon,aerialLost,duelWon,duelLost,dispossessed,totalContest,wonContest,clearanceOffLine,interceptionWon,totalTackle,lastManTackle,fouls,blockedScoringAttempt,challengeLost,ownGoals,errorLeadToAGoal,errorLeadToAShot,penaltyConceded
29,3,Newcastle United,3,3,Manchester City,Erling Haaland,Manchester City,F,False,90.0,7.1,1.0,0.0,1.0,1.0,2.0,3.0,0.0,0.0,1.0,1.0,33.0,0.0,0.0,0.0,2.0,9.0,0.0,2.0,5.0,4.0,1.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29,1,West Ham United,0,2,Manchester City,Erling Haaland,Manchester City,F,False,78.0,7.7,2.0,0.0,0.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
